In [1]:
import torch
import os 
import pandas as pd
import csv
import yaml

In [5]:
def GetDateStr(csv_dict, idx):
    OutStr = ''
    for key in date_rel_dict:
        OutStr += str(csv_dict[key][idx])+'_'
    
    return OutStr[:-1]

date_rel_dict = {'arrival_date_year':True, 'arrival_date_month':True, 'arrival_date_day_of_month':True}

def GetIdxDict(fn):
    t_dict = pd.read_csv(fn)
    dict_date_to_idx = {}
    data_cnt = len(t_dict['arrival_date_year'])
    for i in range(data_cnt): 
        date_str = GetDateStr(t_dict, i)
        if dict_date_to_idx.get(date_str) == None:
            dict_date_to_idx[date_str] = []
        dict_date_to_idx[date_str].append(i)
        
    return dict_date_to_idx

In [8]:
train_dict = GetIdxDict('data/preprocessed/train_processed.csv')
test_dict = GetIdxDict('data/preprocessed/test_processed.csv')
with open('data/preprocessed/test_prep.yaml', 'w') as outfile:
    yaml.dump(test_dict, outfile, default_flow_style=False)
    
with open('data/preprocessed/train_prep.yaml', 'w') as outfile:
    yaml.dump(train_dict, outfile, default_flow_style=False)



[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121]

In [46]:
#funcs
def read_file(fn):
    with open(fn, mode='r') as test:
        reader = csv.reader(test)
        rowList = [row[1:] for row in reader]
        header = rowList.pop(0)
        return header
    
month_dict = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5 ,'June':6,
              'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}

date_rel_dict = {'arrival_date_year':True, 'arrival_date_month':True, 'arrival_date_day_of_month':True}

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def GetDateStr(csv_dict, idx):
    OutStr = ''
    for key in date_rel_dict:
        OutStr += str(csv_dict[key][idx])+'_'
    
    return OutStr[:-1]

def DictIsFloat(dt):
    for key in dt:
        if not isfloat(key):
            return False
    return True   

In [64]:
train_dict = pd.read_csv('data/train.csv').fillna(-1)
test_dict = pd.read_csv('data/test.csv').fillna(-1)
header = read_file('data/test.csv')
header = [hd for hd in header if not date_rel_dict.get(hd)]
float_header = [hd for hd in header if DictIsFloat(train_dict[hd])]
nonfloat_header = [hd for hd in header if not DictIsFloat(train_dict[hd])]
nonfloat_header_dict_dict = {}

for hd in nonfloat_header:
    sub_dict = {}
    for mem in list(train_dict[hd])+list(test_dict[hd]):
        if sub_dict.get(mem) == None:
            sub_dict[mem] = len(sub_dict)
    nonfloat_header_dict_dict[hd] = sub_dict

#check sized for one-hot 
for key in nonfloat_header_dict_dict:
    print(key, len(nonfloat_header_dict_dict[key]))
    
with open('data/preprocessed/feature_encodings.yaml', 'w') as outfile:
    yaml.dump(nonfloat_header_dict_dict, outfile, default_flow_style=False)
    
for key in date_rel_dict:
    float_header.append(key)

train_dict['arrival_date_month'] = list([month_dict[month] for month in list(train_dict['arrival_date_month'])])
test_dict['arrival_date_month'] = list([month_dict[month] for month in list(test_dict['arrival_date_month'])])

test, train = [], []
for hd in nonfloat_header:
    tmp = []
    for mem in test_dict[hd]:
        encoded = nonfloat_header_dict_dict[hd][mem]
        tmp.append(encoded)
    test.append(tmp)
    
    tmp = []
    for mem in train_dict[hd]:
        encoded = nonfloat_header_dict_dict[hd][mem]
        tmp.append(encoded)
    train.append(tmp)
    
for hd in float_header:
    tmp = []
    for mem in test_dict[hd]:
        tmp.append(mem)
    test.append(tmp)
    
    tmp = []
    for mem in train_dict[hd]:
        tmp.append(mem)
    train.append(tmp)
    
with open('data/preprocessed/test.csv', 'w', newline='') as csvfile:
    fieldnames = nonfloat_header+float_header
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rowDict = {}
    for i in range(len(test[0])):
        for idx, fn in enumerate(fieldnames):
            rowDict[fn] = test[idx][i]
        writer.writerow(rowDict)
print('Successfully writen')

with open('data/preprocessed/train.csv', 'w', newline='') as csvfile:
    fieldnames = nonfloat_header+float_header
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rowDict = {}
    for i in range(len(train[0])):
        for idx, fn in enumerate(fieldnames):
            rowDict[fn] = train[idx][i]
        writer.writerow(rowDict)
print('Successfully writen')


hotel 2
meal 5
country 178
market_segment 8
distribution_channel 5
reserved_room_type 10
assigned_room_type 12
deposit_type 3
customer_type 4
Successfully writen
Successfully writen


In [ ]:
with open('data/preprocessed/labels.csv', 'w', newline='') as csvfile:
    fieldnames = ['adr', 'is_canceled']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    rowDict = {}
    for i in range(len(train[0])):
        for fn in fieldnames:
            rowDict[fn] = train[idx][i]
        writer.writerow(rowDict)



In [2]:
#encode non float scripts with one-hot vec and show encoding dictionary
tensors = prep.GetFeatureVectorsAndLabels(fnTrain = 'data/train.csv', fnTest = 'data/test.csv')

Encoding completed

Using the Following Notation: 
 {'Check-Out, 0': 0, 'Check-Out, 1': 1, 'Canceled, 0': 2, 'Canceled, 1': 3, 'No-Show, 0': 4, 'No-Show, 1': 5}


In [3]:
#save tensors
fnList = ['train_data_tensor.pt', 'test_data_tensor.pt', 'train_label_tensor_adr.pt', 'train_label_tensor_rev.pt']
for tensor, fn in zip(tensors, fnList):
    torch.save(tensor, os.path.join('data', fn))
    
#reload to check if saving succeeded
for fn in fnList:
    fn = os.path.join('data', fn)
    print(fn, '\t', torch.load(fn).shape)

data/train_data_tensor.pt 	 torch.Size([91531, 258])
data/test_data_tensor.pt 	 torch.Size([27859, 258])
data/train_label_tensor_adr.pt 	 torch.Size([91531, 1])
data/train_label_tensor_rev.pt 	 torch.Size([91531, 1])


In [4]:
#split test set from train set for validations
train_data_tensor, _,  train_label_tensor_adr, train_label_tensor_rev = tensors
tensors = prep.RandomSplit(train_data_tensor, train_label_tensor_adr, train_label_tensor_rev, 10000)

In [5]:
#save tensors
fnList = ['test/data.pt', 'train/data.pt', 'test/adr.pt', 'train/adr.pt', 'test/rev.pt', 'train/rev.pt']
for tensor, fn in zip(tensors, fnList):
    torch.save(tensor, os.path.join('data', fn))

#reload to check if saving succeeded   
for fn in fnList:
    fn = os.path.join('data', fn)
    print(fn, '\t', torch.load(fn).shape)

data/test/data.pt 	 torch.Size([10000, 258])
data/train/data.pt 	 torch.Size([81531, 258])
data/test/adr.pt 	 torch.Size([10000, 1])
data/train/adr.pt 	 torch.Size([81531, 1])
data/test/rev.pt 	 torch.Size([10000, 1])
data/train/rev.pt 	 torch.Size([81531, 1])


In [ ]:
def GetDataSet(fnTrain, fnTest):
    